In [28]:
#Import necessary libraries

import msal
import requests
import json
import pandas as pd

StatementMeta(SparkPool01, 5, 28, Finished, Available)

In [29]:
#Set parameters

client_id = "bcf8082e-1493-4488-9f90-de22000007a9" #Add your Client ID here
client_secret = "BR0dH_dD693XBX5631_24_t-_KRutM0Avc" #Add your Client Secret here
tenant_name = "kbubalo.com" #Add your tenant name here
workspace_id = "80945c34-5d3f-4998-b561-8f662a6fee46" #Add your Workspace ID here
dataset_id = "b428a792-e384-4fbe-bcba-183492d4912b" #Add your Dataset ID here

authority_url = "https://login.microsoftonline.com/" + tenant_name 
scope = ["https://analysis.windows.net/powerbi/api/.default"]
url = "https://api.powerbi.com/v1.0/myorg/groups/" + workspace_id +"/datasets/"+ dataset_id +"/refreshes?$top=1"

StatementMeta(SparkPool01, 5, 29, Finished, Available)

In [30]:
#Use MSAL to grab token

app = msal.ConfidentialClientApplication(client_id, authority=authority_url, client_credential=client_secret)
result = app.acquire_token_for_client(scopes=scope)

#Get latest Power BI Dataset Refresh
if 'access_token' in result:
    access_token = result['access_token']
    header = {'Content-Type':'application/json', 'Authorization':f'Bearer {access_token}'}
    api_call = requests.get(url=url, headers=header)

    result = api_call.json()['value']
    
    df = pd.DataFrame(result, columns=['requestId', 'id', 'refreshType', 'startTime', 'endTime', 'status'])
    df.set_index('id')

StatementMeta(SparkPool01, 5, 30, Finished, Available)

,requestId,refreshType,startTime,endTime,status
id,,,,,
127702015,00eaeccc-1df5-4b66-b462-154f4d4b51c3,ViaApi,2022-01-05T22:16:47.867Z,2022-01-05T22:17:22.53Z,Completed


In [31]:
if df.status[0] == "Unknown":
    print("Dataset is refreshing right now. Please wait until this refresh has finished to trigger a new one.")
elif df.status[0] == "Disabled":
    print("Dataset refresh is disabled. Please enable it.")
elif df.status[0] == "Failed":
    print("Last Dataset refresh failed. Please check error message.")
elif df.status[0] == "Completed":
    api_call = requests.post(url=url, headers=header)
    print("We triggered a Dataset refresh.")
else:
    print("Not familiar with status, please check documentatino for status: '" + df.status[0] + "'")

StatementMeta(SparkPool01, 5, 31, Finished, Available)

We triggered a Dataset refresh.